<a href="https://colab.research.google.com/github/aaalexlit/omdena_climate_change_challenge_notebooks/blob/main/finetune_multivers_on_climate_fever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune pretrained MultiVerS model on modified Climate-FEVER dataset

Following [these instructions](https://github.com/dwadden/multivers/blob/main/doc/training.md)

The idea is to re-use the code from the instruction but replace the content of `covidfact` folder with the modified CLIMATE-FEVER 

In [1]:
%%capture
!git clone https://github.com/dwadden/multivers.git
!git clone https://github.com/allenai/scifact-evaluator.git
!pip install virtualenv
!virtualenv multivers
!source /content/multivers/bin/activate; pip install -r /content/multivers/requirements.txt

In [2]:
!python /content/multivers/script/get_checkpoint.py longformer_large_science


--2023-04-25 04:38:01--  https://scifact.s3.us-west-2.amazonaws.com/longchecker/latest/checkpoints/longformer_large_science.ckpt
Resolving scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)... 52.92.208.170, 52.218.184.201, 3.5.79.119, ...
Connecting to scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)|52.92.208.170|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1646843693 (1.5G) [binary/octet-stream]
Saving to: ‘checkpoints/longformer_large_science.ckpt’

checkpoints/longfor 100%[===================>]   1.53G  35.4MB/s    in 41s     

2023-04-25 04:38:42 (38.5 MB/s) - ‘checkpoints/longformer_large_science.ckpt’ saved [1646843693/1646843693]



In [3]:
!cp -r /content/checkpoints /content/multivers/checkpoints

In [4]:
# Download the train data.

!gdown https://drive.google.com/uc?id=1uiG3RCA366nbLX9NoEbYX5BAi2NFXiKG
!tar -xf data_train.tar.gz
!rm data_train.tar.gz

!mv /content/data_train /content/multivers/data_train

Downloading...
From: https://drive.google.com/uc?id=1uiG3RCA366nbLX9NoEbYX5BAi2NFXiKG
To: /content/data_train.tar.gz
100% 417M/417M [00:02<00:00, 184MB/s]


In [5]:
%%capture
!pip install datasets
!pip install jsonlines

In [6]:
from datasets import load_dataset
import pandas as pd
import jsonlines

ds_orig = load_dataset("climate_fever", split='test')
df = ds_orig.to_pandas()

mv_label_dict = {0: 'SUPPORT', 1: 'CONTRADICT'}

docs = {}
doc_id = 0
indexed_evidence_sents = set()

counter = 0

with jsonlines.open('claims_comb_train.jsonl', 'w') as claims_train_writer, \
    jsonlines.open('claims_comb_test.jsonl', 'w') as claims_test_writer, \
    jsonlines.open('claims_comb_dev.jsonl', 'w') as claims_dev_writer:
    for claim_row in df.to_dict('records'):
        counter += 1
        evidences = claim_row['evidences']
        cur_evidence_dict = {}
        cur_claim_doc_ids = set()
        for evidence in evidences:
            evidence_label = evidence['evidence_label']
            evidence_article = evidence['article']
            evidence_sent = evidence['evidence']
            if evidence_article in docs:
                doc = docs.get(evidence_article)
            else:
                doc = {"doc_id": doc_id,
                       "title": evidence_article,
                       "abstract": []}
                docs[evidence_article] = doc
                doc_id += 1
            cur_doc_id = doc["doc_id"]
            cur_claim_doc_ids.add(cur_doc_id)
            abstract = doc["abstract"]
            # Don't allow abstracts longer than 4096 otherwise longformer won't work
            if len(''.join(abstract).split()) > 2900:
                continue
            if not evidence_sent in indexed_evidence_sents:
                abstract.append(evidence_sent)
                indexed_evidence_sents.add(evidence_sent)
                sent_ind = len(abstract) - 1
            else:
                sent_ind = abstract.index(evidence_sent)

            # add only evidences to the documents that are not NEI
            if evidence_label != 2:
                curr_sent = {
                    "sentences": [sent_ind],
                    "label": mv_label_dict[evidence_label]
                }
                if f"{cur_doc_id}" in cur_evidence_dict:
                    exist_sents = cur_evidence_dict[f"{cur_doc_id}"]
                    for s in exist_sents:
                        if s['label'] == curr_sent['label']:
                            s['sentences'].append(sent_ind)
                else:
                    cur_evidence_dict[f"{cur_doc_id}"] = [curr_sent]

        claim_doc = {
            'id': int(claim_row['claim_id']),
            'claim': claim_row['claim'],
            'cited_doc_ids': list(cur_claim_doc_ids),
            'evidence': cur_evidence_dict
        }
        if counter % 4 == 0:
            claims_test_writer.write(claim_doc)
        else:
            claims_train_writer.write(claim_doc)
        if counter % 20 == 0:
          claims_dev_writer.write(claim_doc)


with jsonlines.open('corpus_comb_for_training.jsonl', 'w') as corpus_writer:
    for doc in docs.values():
        if doc["abstract"]:
            corpus_writer.write(doc)


Generating test split:   0%|          | 0/1535 [00:00<?, ? examples/s]

Dataset climate_fever downloaded and prepared to /root/.cache/huggingface/datasets/climate_fever/default/1.0.1/9091d2d01d327631e4880d42fda9e8d69da1fb59b53b12d68867cee6e5770ba4. Subsequent calls will reuse this data.


In [7]:
!rm /content/multivers/data_train/target/covidfact/*
!mv /content/claims_comb_test.jsonl /content/multivers/data_train/target/covidfact/claims_test.jsonl
!mv /content/claims_comb_train.jsonl /content/multivers/data_train/target/covidfact/claims_train.jsonl
!mv /content/claims_comb_dev.jsonl /content/multivers/data_train/target/covidfact/claims_dev.jsonl
!mv /content/corpus_comb_for_training.jsonl /content/multivers/data_train/target/covidfact/corpus.jsonl

manually  change line 59 of /content/multivers/script/train_target.py
to become `        "multivers/multivers/train.py",
`

# to continue training
manually change this line https://github.com/dwadden/multivers/blob/main/script/train_target.py#L65 to start from available checkpoint
in /content/checkpoints_user/covidfact/checkpoint/last.ckpt

and lower number of epochs by 1 on ine 51 of the same file

In [8]:
%%writefile /content/multivers/script/train_target.py
"""
Kickoff training on target datasets.

NOTE: Training right now doesn't work with multiple GPU's and DDP. This is known issue;
see for instance https://lightning.ai/forums/t/gradient-checkpointing-ddp-nan/398/7.
"""


import argparse
import subprocess


def get_args():
    help_gpus = """GPU's used for training.
    If a single int, specifies the number of GPU's.
    If a comma-separated list, specifies the specific device ID's.
    For a single specific device, write it as `[device-num],`
    """

    parser = argparse.ArgumentParser("Kick off model training.")
    parser.add_argument(
        "--dataset",
        type=str,
        help="The dataset to train on.",
        choices=["scifact_20", "scifact_10", "healthver", "covidfact"],
    )
    parser.add_argument("--gpus", type=str, help=help_gpus)
    parser.add_argument(
        "--gradient_checkpointing",
        action="store_true",
        help="Turning this on decreases memory usage at the cost of slower training",
    )
    args = parser.parse_args()

    return args


def main():
    args = get_args()
    gpus = args.gpus

    # Deal with case of specific devides
    if "," in gpus:
        n_gpus = len([x for x in gpus.split(",") if x])
    else:
        n_gpus = int(gpus)

    if n_gpus not in [1, 2, 4, 8]:
        raise ValueError("The number of GPU's must be a power of 2.")

    epochs = 3
    workers_per_gpu = 4  # Number of CPU's per gpu.
    effective_batch_size = 8  # Desired effective batch size.
    accumulate_grad_batches = effective_batch_size // n_gpus
    num_workers = workers_per_gpu * n_gpus

    cmd = [
        "python",
        "multivers/multivers/train.py",
        "--result_dir",
        "checkpoints_user",
        "--datasets",
        args.dataset,
        "--starting_checkpoint",
        "/content/checkpoints/last.ckpt",
        "--experiment_name",
        args.dataset,
        "--num_workers",
        num_workers,
        "--gpus",
        gpus,
        "--accumulate_grad_batches",
        accumulate_grad_batches,
        "--lr",
        "1e-5",
        "--precision",
        16,
        "--max_epochs",
        epochs,
        "--scheduler_total_epochs",
        epochs,
        "--train_batch_size",
        1,
        "--eval_batch_size",
        2,
        "--encoder_name",
        "longformer-large-science",
        "--no_reweight_labels",
    ]

    # If training on more than 1 gpu, use DDP accelerator.
    if n_gpus > 1:
        cmd.extend(["--accelerator", "ddp"])

    # Turn on gradient checkpointing if requested.
    if args.gradient_checkpointing:
        cmd.append("--gradient_checkpointing")

    subprocess.call(map(str, cmd))


if __name__ == "__main__":
    main()

Overwriting /content/multivers/script/train_target.py


# To start training

In [38]:
# !source /content/multivers/bin/activate; python /content/multivers/script/train_target.py \
#         --dataset=covidfact --gpus=1 --gradient_checkpointing

Global seed set to 76
fatal: not a git repository (or any of the parent directories): .git
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
/content/multivers/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Experiment logs directory ./checkpoints_user/covidfact exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  warnings.warn(*args, **kwargs)

  | Name                 | Type            | Params
---------------------------------------------------------
0 | encoder              | LongformerModel | 434 M 
1 | dropout              | Dropout         | 0     
2 | label_classifier     | FeedForward     | 1.1 M 
3 | rationale_classifier | FeedForward     | 2.1 M 
4 | metrics              | ModuleDict      | 0     
---------------------------------------------------------
437 M     Trainable params
0         Non-trainable params
437 M     Total pa

In [8]:
# !source /content/multivers/bin/activate; python /content/multivers/script/train_target.py \
#         --dataset=covidfact --gpus=1 --gradient_checkpointing

Global seed set to 76
fatal: not a git repository (or any of the parent directories): .git
Downloading: 100% 803/803 [00:00<00:00, 628kB/s]
Downloading: 100% 899k/899k [00:00<00:00, 5.65MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 3.67MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 12.8MB/s]
Downloading: 100% 1.74G/1.74G [00:45<00:00, 37.9MB/s]
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
/content/multivers/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Experiment logs directory ./checkpoints_user/covidfact_0 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  warnings.warn(*args, **kwargs)

  | Name                 | Type            | Params
---------------------------------------------------------
0 | encoder              | LongformerModel | 434 M 
1 | dropout              | Dropout         | 0     
2 | label_classifier     | 

## Download the last checkpoit 

In [9]:
!gsutil cp gs://cc-evidences-data/multivers_checkpoints/covidfact/checkpoint/last.ckpt /content/checkpoints/last.ckpt

Copying gs://cc-evidences-data/multivers_checkpoints/covidfact/checkpoint/last.ckpt...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

\ [1 files][  4.9 GiB/  4.9 GiB]   23.9 MiB/s                                   
Operation completed over 1 objects/4.9 GiB.                                      


## Continue training

In [11]:
# !source /content/multivers/bin/activate; python /content/multivers/script/train_target.py \
#         --dataset=covidfact --gpus=1 --gradient_checkpointing

Global seed set to 76
fatal: not a git repository (or any of the parent directories): .git
Downloading: 100% 803/803 [00:00<00:00, 501kB/s]
Downloading: 100% 899k/899k [00:00<00:00, 5.13MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 9.87MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 4.34MB/s]
Downloading: 100% 1.74G/1.74G [00:32<00:00, 53.4MB/s]
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
/content/multivers/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Experiment logs directory ./checkpoints_user/covidfact exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  warnings.warn(*args, **kwargs)

  | Name                 | Type            | Params
---------------------------------------------------------
0 | encoder              | LongformerModel | 434 M 
1 | dropout              | Dropout         | 0     
2 | label_classifier     | Fe

In [10]:
!source /content/multivers/bin/activate; python /content/multivers/script/train_target.py \
        --dataset=covidfact --gpus=1 --gradient_checkpointing

Global seed set to 76
fatal: not a git repository (or any of the parent directories): .git
Downloading: 100% 803/803 [00:00<00:00, 676kB/s]
Downloading: 100% 899k/899k [00:00<00:00, 3.45MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 4.93MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 7.24MB/s]
Downloading: 100% 1.74G/1.74G [00:33<00:00, 52.1MB/s]
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.
/content/multivers/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Experiment logs directory ./checkpoints_user/covidfact exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  warnings.warn(*args, **kwargs)

  | Name                 | Type            | Params
---------------------------------------------------------
0 | encoder              | LongformerModel | 434 M 
1 | dropout              | Dropout         | 0     
2 | label_classifier     | Fe

In [11]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=Ib63w28hFX3RdVhCdvT8WZEQ38iOzh&prompt=consent&access_type=offline&code_challenge=C-9-QEu4aCPtlW2ohuVvo0c64RMTNnvNEt9vHP6mh68&code_challenge_method=S256

Enter authorization code: 4/0AVHEtk5gRAQUvM9Rhs2Juz6_zidy0Z9KNgTDmauACy5rxLndxhpj_1Xvv1LOEcoPigQ9Tg

You are now logged in as [aaalex.lit@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [12]:
!gcloud config set project cc-evidence-service

Updated property [core/project].


In [13]:
!gcloud storage cp --recursive /content/checkpoints_user/covidfact gs://cc-evidences-data/multivers_checkpoints

Copying file:///content/checkpoints_user/covidfact/metrics.csv to gs://cc-evidences-data/multivers_checkpoints/covidfact/metrics.csv
Copying file:///content/checkpoints_user/covidfact/checkpoint/epoch=2-step=1319.ckpt to gs://cc-evidences-data/multivers_checkpoints/covidfact/checkpoint/epoch=2-step=1319.ckpt
Copying file:///content/checkpoints_user/covidfact/checkpoint/last.ckpt to gs://cc-evidences-data/multivers_checkpoints/covidfact/checkpoint/last.ckpt
Copying file:///content/checkpoints_user/covidfact/events.out.tfevents.1682397872.c1ef10dbf42a.6926.0 to gs://cc-evidences-data/multivers_checkpoints/covidfact/events.out.tfevents.1682397872.c1ef10dbf42a.6926.0
Copying file:///content/checkpoints_user/covidfact/hparams.yaml to gs://cc-evidences-data/multivers_checkpoints/covidfact/hparams.yaml

Average throughput: 55.1MiB/s
